# From Python to Production — Day 2  
## Notebook 5 — Core Data Structures

By **Prerna Joshi** | #25DaysOfDataTech 

"Choosing the right data structure is half the solution — speed, clarity, and correctness start here."

---

### What you'll learn
- Lists, tuples, sets, and dictionaries — strengths & trade-offs
- Comprehensions and slicing fluency
- Stacks, queues, and priority queues (`deque`, `heapq`)
- Frequency maps (`Counter`) and smart defaults (`defaultdict`)
- Copy semantics: aliasing, shallow vs deep copy
- Practical patterns for real data tasks (grouping, dedup, top‑k)
- Big‑O cheatsheet & performance tips


> **Why this matters for data work**  
> Choosing the right structure often yields bigger wins than micro‑optimizing code. The shape of your data determines complexity, readability, and cost.


## 1. Lists — Fast append, ordered, mutable

- Great general‑purpose sequence
- `append`/`extend`/`pop` from end are **amortized O(1)**
- Costly random inserts/deletes in the middle (O(n))


In [1]:
nums = [10, 20, 30]
nums.append(40)
nums.extend([50, 60])
nums.insert(1, 15)    # O(n)
nums.pop()            # from end O(1) amortized
nums, len(nums), nums[::2]


([10, 15, 20, 30, 40, 50], 6, [10, 20, 40])

## 2. Tuples — Immutable, hashable when elements are hashable

- Use for fixed records, dictionary keys, safe iteration
- Slightly smaller & faster than lists for read‑only data


In [2]:
pt = (12.5, 7.2)
rect = ((0, 0), (3, 4))
hashable_key = ("user", 42)
{hashable_key: "ok"}, pt[0], rect[1]


({('user', 42): 'ok'}, 12.5, (3, 4))

## 3. Sets — Unique membership with average O(1) operations

- Use for deduplication and membership tests
- Unordered; elements must be hashable


In [3]:
users = ["a", "b", "a", "c", "b"]
unique = set(users)            # {'a','b','c'}
fast_lookup = "c" in unique    # O(1) avg
unique, fast_lookup


({'a', 'b', 'c'}, True)

## 4. Dictionaries — Key → Value mapping (insertion‑ordered since Python 3.7+)

- Average O(1) get/set/delete
- Perfect for labeled data, indexes, and grouping


In [4]:
profile = {"id": 101, "name": "Prerna", "role": "Data Engineer"}
profile["skills"] = ["Python", "Pandas"]
profile.get("location", "Unknown"), list(profile.items())


('Unknown',
 [('id', 101),
  ('name', 'Prerna'),
  ('role', 'Data Engineer'),
  ('skills', ['Python', 'Pandas'])])

## 5. Comprehensions — Expressive, fast, readable

- List / dict / set comprehensions
- Add conditions and transforms succinctly


In [5]:
data = [("alice", 91), ("bob", 78), ("carol", 88), ("dave", 95)]
passed = {name: score for name, score in data if score >= 85}
evens = [x for x in range(10) if x % 2 == 0]
letters = {c for c in "abracadabra" if c.isalpha()}
passed, evens, letters


({'alice': 91, 'carol': 88, 'dave': 95},
 [0, 2, 4, 6, 8],
 {'a', 'b', 'c', 'd', 'r'})

## 6. Slicing — Powerful view/copy tool

```
seq[start:stop:step]
```
- O(k) to create a new slice of size k
- `[::-1]` reverses a sequence copy


In [6]:
s = list(range(10))
s1 = s[2:7]
s2 = s[:5]
s3 = s[::-1]
s1, s2, s3


([2, 3, 4, 5, 6], [0, 1, 2, 3, 4], [9, 8, 7, 6, 5, 4, 3, 2, 1, 0])

## 7. Copying & Aliasing — Avoid accidental mutations

```
# Aliasing (same object)
a = [1,2,3]
b = a                # b is a

# Shallow copies
c = a[:]             # list slice copy
d = list(a)          # constructor copy
e = a.copy()

# Deep copy (recursively copies nested objects)
from copy import deepcopy
f = deepcopy(a_nested)
```

**ASCII mental model**

```
Before:
a ─┐
b ─┘  →  [1, 2, 3]

After shallow copy:
c → [1, 2, 3]   (independent top list)

But for nested:
nested = [[1], [2]]
shallow = list(nested)     # inner lists shared!
deep    = deepcopy(nested) # inner lists copied
```


In [7]:
from copy import deepcopy

nested = [[1], [2]]
alias = nested          # same object
shallow = list(nested)  # new outer list, same inner lists
deep = deepcopy(nested) # full recursive copy

nested[0].append(99)
(id(nested), id(alias), id(shallow), id(deep)), nested, shallow, deep


((2010592045888, 2010592045888, 2010592081664, 2010592068096),
 [[1, 99], [2]],
 [[1, 99], [2]],
 [[1], [2]])

## 8. Stacks & Queues

- **Stack**: use list `.append()` and `.pop()` (end)  
- **Queue/Deque**: use `collections.deque` for O(1) appends/pops from both ends


In [8]:
from collections import deque

stack = []
stack.append(1); stack.append(2); stack.append(3)
last = stack.pop()

q = deque()
q.append("A"); q.append("B"); q.append("C")
left = q.popleft()   # O(1)
stack, last, list(q), left


([1, 2], 3, ['B', 'C'], 'A')

## 9. Priority Queues — `heapq` (min‑heap)

- Push/pop are O(log n)
- For max‑heap behavior, push negatives or use tuples `(priority, item)`


In [9]:
import heapq

scores = [92, 88, 75, 96, 81]
heap = []
for s in scores:
    heapq.heappush(heap, s)

top1 = heapq.nlargest(1, scores)[0]
pop_smallest = heapq.heappop(heap)
top3 = heapq.nlargest(3, scores)
top1, pop_smallest, top3


(96, 75, [96, 92, 88])

## 10. Frequency & Defaults — `Counter` and `defaultdict`

- `Counter` builds frequency maps fast
- `defaultdict(list)` simplifies grouping without `KeyError` checks


In [10]:
from collections import Counter, defaultdict

words = "to be or not to be that is the question".split()
freq = Counter(words)
groups = defaultdict(list)
for name, score in [("alice", 91), ("bob", 78), ("carol", 88), ("alice", 95)]:
    groups[name].append(score)

freq.most_common(3), dict(groups)


([('to', 2), ('be', 2), ('or', 1)],
 {'alice': [91, 95], 'bob': [78], 'carol': [88]})

## 11. Practical Patterns for Data Work

- **Dedup while preserving order**  
  Use a seen set and append unseen.
- **Grouping**  
  Use `defaultdict(list)` or `itertools.groupby` (sorted).
- **Top‑k**  
  Use `heapq.nlargest(k, data)` without sorting everything.
- **Index by key**  
  Build `{row["id"]: row for row in rows}` for O(1) lookup.


In [11]:
def dedup_preserve_order(seq):
    seen = set()
    out = []
    for x in seq:
        if x not in seen:
            seen.add(x)
            out.append(x)
    return out

rows = [
    {"id": 2, "name": "b"},
    {"id": 1, "name": "a"},
    {"id": 2, "name": "b2"},
]
index = {r["id"]: r for r in rows}   # last wins
dedup_preserve_order([1,2,1,3,2,4]), index


([1, 2, 3, 4], {2: {'id': 2, 'name': 'b2'}, 1: {'id': 1, 'name': 'a'}})

## 12. Big‑O Cheatsheet (Common Ops)

| Structure | Access | Search | Insert | Delete | Notes |
|---|---:|---:|---:|---:|---|
| List (end) | O(1) | O(n) | O(1)* | O(1)* | *Amortized at end; middle ops O(n) |
| Tuple | O(1) | O(n) | — | — | Immutable |
| Set | — | **O(1)** | **O(1)** | **O(1)** | Average‑case |
| Dict | — | **O(1)** | **O(1)** | **O(1)** | Average‑case |
| Deque | — | — | **O(1)** ends | **O(1)** ends | Double‑ended |
| Heap | — | — | O(log n) | O(log n) | Min‑heap by default |

> **Tip:** Prefer `set`/`dict` for membership and indexing; prefer `deque` for queues; use `heapq` for streaming top‑k.


## 13. Common Pitfalls & How to Avoid Them

- **Aliasing surprises**: copying references, not objects → use `copy()` or `deepcopy()` when needed.  
- **Mutable keys**: dict/set keys must be hashable (immutable).  
- **Sorting mixed types**: define a key to avoid `TypeError`.  
- **Overusing lists**: for membership checks, switch to `set` for O(1) average time.


## 14. Practice (Try first, then reveal solutions)

1. **dedup**: Remove duplicates from a list but preserve input order.  
2. **group_by_first_letter**: Given a list of words, group them by first letter using `defaultdict(list)`.  
3. **top_k**: Return the top‑`k` largest numbers from a stream using `heapq`.  
4. **word_freq**: Build a frequency dict for words, return the top 5.  
5. **flatten_unique**: Given a nested list of integers (depth ≤ 2), return a **set** of unique values.  
6. **index_by**: Convert a list of dicts to a dict keyed by a given field (e.g., `"id"`).  
7. **stable_sort_by_len**: Sort strings by length ascending; ties maintain original order.  
8. **safe_get**: Given nested dicts, implement dotted path lookup `"a.b.c"` with default.  
9. **deque_window_max**: Use `deque` to maintain a sliding window of size `w` and report the **current max** each step.  
10. **deepcopy_demo**: Show with code how shallow copy differs from deep copy on a nested list.  
11. **counter_diff**: Using `Counter`, compute words appearing more in `A` than in `B`.  
12. **two_sum_indices**: Given a list and a target, return indices of two numbers summing to target using a dict for O(n).


## 15. Practice Solutions  
*(Click to reveal after solving.)*

<details>
<summary><strong>Solution 1️⃣ — dedup</strong></summary>

```python
def dedup(seq):
    seen = set()
    out = []
    for x in seq:
        if x not in seen:
            seen.add(x)
            out.append(x)
    return out
```
</details>

<details>
<summary><strong>Solution 2️⃣ — group_by_first_letter</strong></summary>

```python
from collections import defaultdict

def group_by_first_letter(words):
    g = defaultdict(list)
    for w in words:
        if w:
            g[w[0]].append(w)
    return dict(g)
```
</details>

<details>
<summary><strong>Solution 3️⃣ — top_k</strong></summary>

```python
import heapq

def top_k(stream, k=3):
    return heapq.nlargest(k, stream)
```
</details>

<details>
<summary><strong>Solution 4️⃣ — word_freq</strong></summary>

```python
from collections import Counter

def word_freq(words, top=5):
    c = Counter(words)
    return c.most_common(top)
```
</details>

<details>
<summary><strong>Solution 5️⃣ — flatten_unique</strong></summary>

```python
def flatten_unique(nested):
    out = set()
    for x in nested:
        if isinstance(x, list):
            out.update(x)
        else:
            out.add(x)
    return out
```
</details>

<details>
<summary><strong>Solution 6️⃣ — index_by</strong></summary>

```python
def index_by(rows, key="id"):
    return {r[key]: r for r in rows}
```
</details>

<details>
<summary><strong>Solution 7️⃣ — stable_sort_by_len</strong></summary>

```python
def stable_sort_by_len(strings):
    return sorted(strings, key=len)  # Python sort is stable
```
</details>

<details>
<summary><strong>Solution 8️⃣ — safe_get</strong></summary>

```python
def safe_get(d, path, default=None):
    cur = d
    for part in path.split("."):
        if isinstance(cur, dict) and part in cur:
            cur = cur[part]
        else:
            return default
    return cur
```
</details>

<details>
<summary><strong>Solution 9️⃣ — deque_window_max</strong></summary>

```python
from collections import deque

def deque_window_max(nums, w):
    q = deque()  # store indices, keep values in decreasing order
    out = []
    for i, x in enumerate(nums):
        while q and nums[q[-1]] <= x:
            q.pop()
        q.append(i)
        if q[0] <= i - w:
            q.popleft()
        if i >= w - 1:
            out.append(nums[q[0]])
    return out
```
</details>

<details>
<summary><strong>Solution 🔟 — deepcopy_demo</strong></summary>

```python
from copy import deepcopy

nested = [[1], [2]]
shallow = list(nested)
deep = deepcopy(nested)
nested[0].append(99)
# shallow reflects inner change, deep does not
result = (nested, shallow, deep)
```
</details>

<details>
<summary><strong>Solution 1️⃣1️⃣ — counter_diff</strong></summary>

```python
from collections import Counter

def counter_diff(A, B):
    ca, cb = Counter(A), Counter(B)
    diff = ca - cb  # subtract counts, floor at 0
    return dict(diff)
```
</details>

<details>
<summary><strong>Solution 1️⃣2️⃣ — two_sum_indices</strong></summary>

```python
def two_sum_indices(nums, target):
    pos = {}
    for i, x in enumerate(nums):
        need = target - x
        if need in pos:
            return pos[need], i
        pos[x] = i
    return None
```
</details>


## 16. Mini Cheatsheet

- Prefer `dict`/`set` for O(1) membership & indexing
- Use `deque` for queue‑like operations (append/pop both ends)
- Use `heapq.nlargest(k, seq)` for top‑k without full sort
- Be explicit about **copying** vs **aliasing**
- Aim for comprehensions over loops when it improves clarity
